# Data Cleaning

In [491]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from geopy import Nominatim

Read both .csv files for cleaning
* Ames Real Estate Data = real_estate
* Amex_HousePrice = housing

In [492]:
housing = pd.read_csv('./data/Ames_Housing_Price_Data.csv', index_col= 0)
real_estate = pd.read_csv('./data/Ames_Real_Estate_data.csv' , index_col = 1)

C:\Users\David Jhang\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Show max columns so we can see everything

In [493]:
#set columns
pd.get_option("display.max_columns") #this will display what the current setting is
pd.set_option("display.max_columns",None)

#set rows
pd.set_option("display.max_rows",None)

Splitter out just Normal sales which was the majority of the data

In [494]:
housing = housing[housing['SaleCondition'] == 'Normal']

## Replace all null values based on context
- replacing metrics instead of removing
- These basements had no basements
- Garages had no garages
- no Fireplaces, No Pool, No Fence, No Alley and No MiscFeatures substitued for NAs

In [495]:
## LotFrontage -- replace with mean of the column 
housing['LotFrontage'].fillna(value = housing['LotFrontage'].mean(), inplace = True)

## Replace Nulls where NA just means that the house does not have the feature 

## Basement metrics
housing['BsmtQual'].fillna(value = 'No_Basement', inplace=True)
housing['BsmtCond'].fillna(value = 'No_Basement', inplace=True)
housing['BsmtExposure'].fillna(value = 'No_Basement', inplace=True)
housing['BsmtFinType1'].fillna(value = 'No_Basement', inplace=True)
housing['BsmtFinType2'].fillna(value = 'No_Basement', inplace=True)

## Garage Type 
housing['GarageType'].fillna(value = 'No_Garage', inplace=True)
housing['GarageYrBlt'].fillna(value = 'No_Garage', inplace=True)
housing['GarageFinish'].fillna(value = 'No_Garage', inplace=True)
housing['GarageQual'].fillna(value = 'No_Garage', inplace=True)
housing['GarageCond'].fillna(value = 'No_Garage', inplace=True)

## Replace other nulls where null just means the feature is not there 
housing['FireplaceQu'].fillna(value = 'No_Fireplace', inplace=True)
housing['PoolQC'].fillna(value = 'No_Pool', inplace=True)
housing['Fence'].fillna(value = 'No_Fence', inplace=True)
housing['MiscFeature'].fillna(value = 'No_Misc', inplace=True)
housing['Alley'].fillna(value = 'No_alley', inplace=True)

## Replacing nulls with 0s
- Replaced values with 0
- assumption is no basements = no measurements or bathrooms
- no Garage = no area or # of cars

In [496]:
housing['BsmtFinSF1'].fillna(value = 0, inplace=True)
housing['BsmtFinSF2'].fillna(value = 0, inplace=True)
housing['BsmtUnfSF'].fillna(value = 0, inplace=True)
housing['MasVnrType'].fillna(value = 0, inplace=True)
housing['TotalBsmtSF'].fillna(value = 0, inplace=True)
housing['MasVnrArea'].fillna(value = 0, inplace=True)
housing['BsmtFullBath'].fillna(value = 0, inplace=True)
housing['BsmtHalfBath'].fillna(value = 0, inplace=True)
housing['GarageCars'].fillna(value = 0, inplace=True)
housing['GarageArea'].fillna(value = 0, inplace=True)

- At this point, only one null value remains in the "Electric Column". We will just remove that one row 

In [497]:
housing.dropna(axis = 0, inplace = True)

In [498]:
housing.isnull().sum() #looks good

PID              0
GrLivArea        0
SalePrice        0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
Alley            0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
MasVnrArea       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinSF1       0
BsmtFinType2     0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr

In [499]:
housing.shape #started with (2580, 81)

(2413, 81)

## Merging Addresses into housing DF
- remove duplicates from housing by PID (unique identifier) if sale is the same year
- will use later to create a heat map
- will add lattitude and longetitude

In [500]:
# checking for duplicate PIDs
housing.PID.duplicated().sum()

1

In [501]:
#locate the row and find the PID
duplicateRowsDF = housing[housing.duplicated()]
print("Duplicate Rows except first occurence based on all columns are:")
print(duplicateRowsDF)

Duplicate Rows except first occurence based on all columns are:
         PID  GrLivArea  SalePrice  MSSubClass MSZoning  LotFrontage  LotArea  \
1  909276070       1717     194000          50       RL         80.0    12400   

  Street     Alley LotShape LandContour Utilities LotConfig LandSlope  \
1   Pave  No_alley      Reg         HLS    AllPub    Inside       Mod   

  Neighborhood Condition1 Condition2 BldgType HouseStyle  OverallQual  \
1      Crawfor       Norm       Norm     1Fam     1.5Fin            5   

   OverallCond  YearBuilt  YearRemodAdd RoofStyle RoofMatl Exterior1st  \
1            6       1940          1950     Gable  CompShg     Wd Sdng   

  Exterior2nd MasVnrType  MasVnrArea ExterQual ExterCond Foundation BsmtQual  \
1     Wd Sdng       None         0.0        TA        TA     CBlock       Gd   

  BsmtCond BsmtExposure BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  \
1       TA           Mn          BLQ       602.0          Unf         0.0   

   BsmtUnfSF  

In [502]:
# check PID = 909276070
housing[housing.PID == 909276070]

# both from 2006, can remove

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
817,909276070,1717,194000,50,RL,80.0,12400,Pave,No_alley,Reg,HLS,AllPub,Inside,Mod,Crawfor,Norm,Norm,1Fam,1.5Fin,5,6,1940,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,Gd,TA,Mn,BLQ,602.0,Unf,0.0,299.0,901.0,GasA,TA,Y,SBrkr,1125,592,0,0.0,0.0,1,1,2,1,TA,7,Typ,1,Gd,Attchd,1940,Unf,1.0,410.0,TA,TA,Y,0,0,0,0,113,0,No_Pool,No_Fence,No_Misc,0,2,2006,WD,Normal
1,909276070,1717,194000,50,RL,80.0,12400,Pave,No_alley,Reg,HLS,AllPub,Inside,Mod,Crawfor,Norm,Norm,1Fam,1.5Fin,5,6,1940,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,Gd,TA,Mn,BLQ,602.0,Unf,0.0,299.0,901.0,GasA,TA,Y,SBrkr,1125,592,0,0.0,0.0,1,1,2,1,TA,7,Typ,1,Gd,Attchd,1940,Unf,1.0,410.0,TA,TA,Y,0,0,0,0,113,0,No_Pool,No_Fence,No_Misc,0,2,2006,WD,Normal


In [503]:
# Removing duplicates
housing = housing.drop_duplicates(subset='PID', keep='first', ignore_index='True')

In [504]:
housing.shape #this should be the final list

(2412, 81)

## MERGING with JUNG's GOOGLE API
- was much more accurate and better than using NOMINATIM
- left the other code below for reference

In [505]:
#import his file
coord = pd.read_csv('./data/housinglatlong.csv', index_col= 0)

In [506]:
# only going to use these columns to merge
coord = coord[['PID', 'Coordinates','Lat','Lon']]

# now merge on housing by PID and MapRefNo
housing = housing.merge(coord, how ='left',
                       on='PID')

In [507]:
housing.head() #checking

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Coordinates,Lat,Lon
0,909176150,856,126000,30,RL,68.217524,7890,Pave,No_alley,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,No_Pool,No_Fence,No_Misc,0,3,2010,WD,Normal,"436 Hayward Ave, Ames, IA 50014, USA",42.017780,-93.651452
1,905476230,1049,139500,120,RL,42.000000,4235,Pave,No_alley,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,No_Fireplace,Attchd,1984,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,No_Pool,No_Fence,No_Misc,0,2,2009,WD,Normal,"3416 West St, Ames, IA 50014, USA",42.024697,-93.664186
2,911128020,1001,124900,30,C (all),60.000000,6060,Pave,No_alley,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,No_Fireplace,Detchd,1930,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,No_Pool,No_Fence,No_Misc,0,11,2007,WD,Normal,"320 S 2nd St, Ames, IA 50010, USA",42.021389,-93.614855
3,535377150,1039,114000,70,RL,80.000000,8146,Pave,No_alley,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,No_Fireplace,Detchd,1940,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,No_Pool,No_Fence,No_Misc,0,5,2009,WD,Normal,"1524 Douglas Ave, Ames, IA 50010, USA",42.038070,-93.612065
4,534177230,1665,227000,60,RL,70.000000,8400,Pave,No_alley,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,8,6,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0,GasA,Ex,Y,SBrkr,810,855,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,No_Fireplace,Attchd,2001,Fin,2.0,528.0,TA,TA,Y,0,45,0,0,0,0,No_Pool,No_Fence,No_Misc,0,11,2009,WD,Normal,"2304 Fillmore Ave, Ames, IA 50010, USA",42.044900,-93.631893


In [508]:
housing.shape # (2412, 81) previously.. looks good

(2412, 84)

## Saving file to new .csv
- named "housing.csv"

In [509]:
# create new cleaned .csv file
housing.to_csv('housing.csv')

## Getting latitude and longitude (DO NOT USE)
- using GEOPY
- some addresses are houses that are split, need to fix these
- create seperate lat and long using the addresses

going to use JUNG's Google Map mapping instead, it's more accurate

In [394]:
# now merge on housing by PID and MapRefNo
housing = housing.merge(real_estate[['MapRefNo','Prop_Addr']], how ='left',
                       left_on='PID', right_on='MapRefNo').drop(columns=['MapRefNo'])

# There are duplicates in the real_estate DF
# Need to clear or it will create duplicate rows in the housing DF after merge
housing = housing.drop_duplicates(subset='PID', keep='first', ignore_index='True')

# create duplicate column for Prop_Addr
housing['Address'] = housing['Prop_Addr']

# replace these addresses that have different units in the same house
housing.loc[housing.Address == '3411 TRIPP ST 3413', 'Address'] = '3411 TRIPP ST'
housing.loc[housing.Address == '3412 TRIPP ST 3414', 'Address'] = '3412 TRIPP ST'
housing.loc[housing.Address == '520 20TH ST 522', 'Address'] = '520 20TH ST'
housing.loc[housing.Address == '2159 COUNTRY CLUB BLVD 2157', 'Address'] = '2159 COUNTRY CLUB BLVD'
housing.loc[housing.Address == '1505 LITTLE BLUESTEM CT UNIT 116', 'Address'] = '1505 LITTLE BLUESTEM CT'
housing.loc[housing.Address == '3901 QUEBEC ST 3903', 'Address'] = '3901 QUEBEC ST'
housing.loc[housing.Address == '709 CLARK AVE 709 1/2', 'Address'] = '709 CLARK AVE'
housing.loc[housing.Address == '916 WILSON AVE 918', 'Address'] = '916 WILSON AVE'
housing.loc[housing.Address == '1505 LITTLE BLUESTEM CT UNIT 119', 'Address'] = '1505 LITTLE BLUESTEM CT'
housing.loc[housing.Address == '3310 OPAL DR 3312', 'Address'] = '3310 OPAL DR'
housing.loc[housing.Address == '1505 LITTLE BLUESTEM CT UNIT 103', 'Address'] = '1505 LITTLE BLUESTEM CT'
housing.loc[housing.Address == '803 13TH ST 805', 'Address'] = '803 13TH ST'
housing.loc[housing.Address == '803 YUMA AVE 805', 'Address'] = '803 YUMA AVE'
housing.loc[housing.Address == '1505 LITTLE BLUESTEM CT UNIT 107', 'Address'] = '1505 LITTLE BLUESTEM CT'
housing.loc[housing.Address == '104 E 20TH ST 106', 'Address'] = '104 E 20TH ST'
housing.loc[housing.Address == '407 JEFFREY LN 409', 'Address'] = '407 JEFFREY LN'
housing.loc[housing.Address == '250 CAMPUS AVE 248', 'Address'] = '250 CAMPUS AVE'
housing.loc[housing.Address == '1505 LITTLE BLUESTEM CT UNIT 106', 'Address'] = '1505 LITTLE BLUESTEM CT'
housing.loc[housing.Address == '615 DULUTH ST 617', 'Address'] = '615 DULUTH ST'
housing.loc[housing.Address == '513 GARDEN RD 515', 'Address'] = '513 GARDEN RD'
housing.loc[housing.Address == '1505 LITTLE BLUESTEM CT UNIT 114', 'Address'] = '1505 LITTLE BLUESTEM CT'
housing.loc[housing.Address == '1121 TOP O HOLLOW RD 1123', 'Address'] = '1121 TOP O HOLLOW RD'
housing.loc[housing.Address == '1505 LITTLE BLUESTEM CT UNIT 112', 'Address'] = '1505 LITTLE BLUESTEM CT'
housing.loc[housing.Address == '2144 SUNSET DR 2146', 'Address'] = '2144 SUNSET DR'
housing.loc[housing.Address == '912 GRAND AVE 914', 'Address'] = '912 GRAND AVE'
housing.loc[housing.Address == '2509 STORM ST 2511', 'Address'] = '2509 STORM ST'
housing.loc[housing.Address == '219 WILMOTH AVE 221', 'Address'] = '219 WILMOTH AVE'
housing.loc[housing.Address == '3304 OPAL DR 3306', 'Address'] = '3304 OPAL DR'
housing.loc[housing.Address == '743 GARNET DR 745', 'Address'] = '743 GARNET DR'
housing.loc[housing.Address == '2140 SUNSET DR 2142', 'Address'] = '2140 SUNSET DR'

# Need to add 'AMES IOWA' to specify state and city
housing['Address'] = housing['Address'] + " AMES IOWA"

# this will pull in lat and long into the housing DF
### DON'T RUN THIS CODE, WILL TAKE AN HOUR! ###
geolocator = Nominatim(user_agent="ram", timeout=10000)

housing['lat'] = housing['Address'].apply(geolocator.geocode).apply(lambda x: x.latitude if x else None)
housing['long'] = housing['Address'].apply(geolocator.geocode).apply(lambda x: x.longitude if x else None)